In [708]:
#import libraries
import pandas as pd
import os
import re

# Find bus stops that will not have change to route names, EOLs, numbers.

## Build table of existing stops from decal
### Format: |Abbr (int), ["Rte_EOL1", etc], "EXISTING"|

In [709]:
# Import decal sheet

current_directory = os.getcwd()
#file_path = os.path.join(current_directory,'BNR_EOL.csv')
file_path = 'Decal_20231130_simple.csv'


# Check if the file exists before reading it
if os.path.exists(file_path):
    # using Pandas to read a CSV file
    existCSV_df = pd.read_csv(file_path)
else:
    print("File not found in the current directory.")

existCSV_df.head()

,StopAbbr,BusStopType,LineAbbr,IsTP,IsEOL,Helper-0,Helper-1,Helper-2,Helper-3,Helper-4,...,EOL-4,EOL-5,EOL-6,EOL-7,EOL-8,EOL-9,EOL-10,EOL-11,EOL-12,CountEOL
0,100004,STR,1;68,FALSE,FALSE,100004-68-Westbound-6,100004-1-Southbound-28,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,100006,STR,1;68,FALSE,FALSE,100006-1-Southbound-29,100006-68-Westbound-7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,100008,SGN,1;68,FALSE,FALSE,100008-68-Westbound-8,100008-1-Southbound-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,100010,STR,1;68,FALSE,FALSE,100010-68-Westbound-9,100010-1-Southbound-31,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,100012,SGN,1;68,FALSE,FALSE,100012-68-Westbound-10,100012-1-Southbound-32,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [710]:
# Identifying all "Helper-" and "EOL-" columns
helper_columns = [col for col in existCSV_df.columns if "Helper-" in col]
eol_columns = [col for col in existCSV_df.columns if "EOL-" in col]

# Modify Helper- cols by extracting route code
def extract_route(s):
    if pd.isna(s):
        return s
    parts = s.split('-')
    return parts[1]

for col in helper_columns:
    existCSV_df[col] = existCSV_df[col].apply(extract_route)

existCSV_df.head()

,StopAbbr,BusStopType,LineAbbr,IsTP,IsEOL,Helper-0,Helper-1,Helper-2,Helper-3,Helper-4,...,EOL-4,EOL-5,EOL-6,EOL-7,EOL-8,EOL-9,EOL-10,EOL-11,EOL-12,CountEOL
0,100004,STR,1;68,FALSE,FALSE,68,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,100006,STR,1;68,FALSE,FALSE,1,68,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,100008,SGN,1;68,FALSE,FALSE,68,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,100010,STR,1;68,FALSE,FALSE,68,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,100012,SGN,1;68,FALSE,FALSE,68,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [711]:
# create list of route_EOL
def concat_routeEOL(row):
    list = []
    for helper, eol in zip(helper_columns, eol_columns):
        helper_val = row[helper]
        eol_val = row[eol]
        if not pd.isna(helper_val) and not pd.isna(eol_val):
            list.append(f"{helper_val}_{eol_val}")
    list.sort()
    return list

In [712]:
# create and populate new table with transformed df
abbrs = existCSV_df['StopAbbr']
exist_df = pd.DataFrame(abbrs)

In [713]:
# apply concat_routeEOL to new df
exist_df["Route_EOL_exist"] = existCSV_df.apply(concat_routeEOL, axis=1)

# apply concat_routeNum to new df
#exist_df["Route_Num_existing"] = existCSV_df.apply(concat_routeNum, axis=1)

In [714]:
def routeNum(row):
    list = []
    routes = row.iloc[1]
    #print(routes)
    for route in routes:
        num = route.split('_')[0]
        list.append(num)
    list.sort()
    return list

In [715]:
exist_df["Route_Num_exist"] = exist_df.apply(routeNum, axis=1)

In [716]:
exist_df.head()

,StopAbbr,Route_EOL_exist,Route_Num_exist
0,100004,"[1_HE Holmes Stn, 68_West End Stn]","[1, 68]"
1,100006,"[1_West End Stn, 68_HE Holmes Stn]","[1, 68]"
2,100008,"[1_HE Holmes Stn, 68_West End Stn]","[1, 68]"
3,100010,"[1_HE Holmes Stn, 68_West End Stn]","[1, 68]"
4,100012,"[1_HE Holmes Stn, 68_West End Stn]","[1, 68]"


## Build table of stops w/ route & EOL (BNR)

In [717]:
#Import table of stops from BNR from Remix

current_directory = os.getcwd()
file_path = 'BNR_Stops.csv'

# Check if the file exists before reading it
if os.path.exists(file_path):
    # For example, using Pandas to read a CSV file
    bnrCSV_df = pd.read_csv(file_path)
else:
    print("File not found in the current directory.")

bnrCSV_df.head()

,StopAbbr,Lines
0,213832,193 Morrow / Jonesboro (Inbound)
1,213831,193 Morrow / Jonesboro (Outbound)
2,213830,"21 Memorial Drive (Inbound), 22 Glenwood (Inbo..."
3,213829,154 Fairburn Heights (Outbound)
4,213825,184 Buffington (Inbound)


In [718]:
# Function to process the "Lines" column and extract "routeCode_Direction"
def extract_route_direction(line):
    # Split the line into separate routes
    routes = line.split(', ')
    formatted_routes = []
    for route in routes:
        # Extract route code and direction
        route_parts = route.split(' ')
        direction_part = route_parts[-1]
        route_code = ''.join(filter(str.isalnum, route_parts[0]))  # Keep only alphanumeric characters
        # Check if direction is correctly enclosed in parentheses
        if direction_part == "(Inbound&Outbound)":
            formatted_routes.append([route_code,"Inbound"])
            formatted_routes.append([route_code,"Outbound"])
        elif direction_part.startswith('(') and direction_part.endswith(')'):
            direction = direction_part.strip('()')
            # Combine route code and direction
            formatted_routes.append([route_code, direction])
    return formatted_routes


In [719]:

# create and populate new table with transformed df
abbrs = bnrCSV_df['StopAbbr']
bnr_df = pd.DataFrame(abbrs)
# Apply the function to the 'Lines' column to create a new column 'Route_Direction'
bnr_df['Route_Direction'] = bnrCSV_df['Lines'].apply(extract_route_direction)

# Displaying the first few rows of the DataFrame with the new column
bnr_df.head()

,StopAbbr,Route_Direction
0,213832,"[[193, Inbound]]"
1,213831,"[[193, Outbound]]"
2,213830,"[[21, Inbound], [22, Inbound]]"
3,213829,"[[154, Outbound]]"
4,213825,"[[184, Inbound]]"


In [720]:
#Import table of route EOLs

current_directory = os.getcwd()
file_path = 'BNR_EOL.csv'

# Check if the file exists before reading it
if os.path.exists(file_path):
    # For example, using Pandas to read a CSV file
    EOL_df = pd.read_csv(file_path)
else:
    print("File not found in the current directory.")

In [721]:
EOL_df.head()

,Route,RouteNum,Dir,EOL
0,1,1,Inbound,West End Stn
1,1,1,Outbound,Arts Center Stn
2,2,2,Inbound,Candler Park Stn
3,2,2,Outbound,Bankhead Stn
4,2L,2,Inbound,Candler Park Stn


In [722]:
def mapEOL(row):
    pattern = r'^\d{1,3}'
    list = []
    #print(row)
    for route in row["Route_Direction"]:
        #print(route)
        rte = re.match(pattern, route[0]).group(0)
        #print([route[0],route[1]])
        #print(df[(df["Route"] == route[0]) &
        #       (df["Dir"] == route[1])].iat[0,3])
        eol = EOL_df[(EOL_df["Route"] == route[0]) &
               (EOL_df["Dir"] == route[1])].iat[0,3]
        list.append(rte + '_' + eol)
    list.sort()
    return list

In [723]:
bnr_df["Route_EOL_bnr"] = bnr_df.apply(mapEOL, axis=1)
bnr_df = bnr_df.drop('Route_Direction', axis=1)


In [724]:
bnr_df.head()

,StopAbbr,Route_EOL_bnr
0,213832,[193_East Point Stn]
1,213831,[193_Justice Center]
2,213830,"[21_West Lake Stn, 22_Ashview Triangle]"
3,213829,[154_HE Holmes Stn]
4,213825,[184_College Park Stn]


In [725]:
bnr_df["Route_Num_bnr"] = bnr_df.apply(routeNum, axis=1)


In [726]:
bnr_df.head()

,StopAbbr,Route_EOL_bnr,Route_Num_bnr
0,213832,[193_East Point Stn],[193]
1,213831,[193_Justice Center],[193]
2,213830,"[21_West Lake Stn, 22_Ashview Triangle]","[21, 22]"
3,213829,[154_HE Holmes Stn],[154]
4,213825,[184_College Park Stn],[184]


## Compare Dataframe of existing bus stops with that of BNR bus stops

In [727]:
# Perform a join from bnr_df onto exist_df
key = 'StopAbbr'
merged_df = pd.merge(exist_df, bnr_df, on=key, how='left', indicator=True)

merged_df.head()


,StopAbbr,Route_EOL_exist,Route_Num_exist,Route_EOL_bnr,Route_Num_bnr,_merge
0,100004,"[1_HE Holmes Stn, 68_West End Stn]","[1, 68]",[1_West End Stn],[1],both
1,100006,"[1_West End Stn, 68_HE Holmes Stn]","[1, 68]",[1_West End Stn],[1],both
2,100008,"[1_HE Holmes Stn, 68_West End Stn]","[1, 68]",[1_West End Stn],[1],both
3,100010,"[1_HE Holmes Stn, 68_West End Stn]","[1, 68]","[1_West End Stn, 22_Ashview Triangle]","[1, 22]",both
4,100012,"[1_HE Holmes Stn, 68_West End Stn]","[1, 68]",[1_West End Stn],[1],both


In [728]:
# add new col with comparison
merged_df["Unchanged_Num"] = merged_df['Route_Num_exist'] == merged_df['Route_Num_bnr']
merged_df["Unchanged_EOL"] = merged_df['Route_EOL_exist'] == merged_df['Route_EOL_bnr']

merged_df.head()

,StopAbbr,Route_EOL_exist,Route_Num_exist,Route_EOL_bnr,Route_Num_bnr,_merge,Unchanged_Num,Unchanged_EOL
0,100004,"[1_HE Holmes Stn, 68_West End Stn]","[1, 68]",[1_West End Stn],[1],both,False,False
1,100006,"[1_West End Stn, 68_HE Holmes Stn]","[1, 68]",[1_West End Stn],[1],both,False,False
2,100008,"[1_HE Holmes Stn, 68_West End Stn]","[1, 68]",[1_West End Stn],[1],both,False,False
3,100010,"[1_HE Holmes Stn, 68_West End Stn]","[1, 68]","[1_West End Stn, 22_Ashview Triangle]","[1, 22]",both,False,False
4,100012,"[1_HE Holmes Stn, 68_West End Stn]","[1, 68]",[1_West End Stn],[1],both,False,False


In [729]:
# remove brackets from lists for export
merged_df['Route_EOL_exist'] = merged_df['Route_EOL_exist'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
merged_df['Route_EOL_bnr'] = merged_df['Route_EOL_bnr'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)

In [730]:
# write merged df to csv
merged_df.to_csv('BSRU_comparison.csv', index=False)